In [20]:
from mako.template import Template
import argparse
import configparser
from itertools import chain
import re

/opt/conda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:6: DeprecationWarning: the sets module is deprecated


In [42]:
DEBUG = True
IMAGE_NAME = 'quagga'

In [3]:
config = configparser.ConfigParser()

In [4]:
# reads lab.conf
with open('shared/MARACAS_lab/lab.conf') as stream:
    # adds a section to mimic a .ini file
    stream = chain(("[dummysection]",), stream)
    config.read_file(stream)

In [21]:
# gets 2 list of keys, one for machines and the other for the metadata
# we also need a unique list of links
keys = []
m_keys = []
links = []
for key in config['dummysection']: 
    if DEBUG: print(key, config['dummysection'][key])
    if '[' in key and ']' in key:
        keys.append(key)
        links.append(config['dummysection'][key])
    else:
        m_keys.append(key)

('lab_description', u'absaaaaaaaaaaaaaaaaa')
('lab_version', u'1.0')
('lab_author', u'aaaaaaaaaaaaaaaa')
('lab_email', u'aaaaaaaaa@aaaaaaaaaaaaaa')
('host[0]', u'A')
('server[0]', u'B')
('ospf1[0]', u'C')
('ospf1[1]', u'E')
('ospf1[2]', u'F')
('ospf2[0]', u'D')
('ospf2[1]', u'E')
('ospf3[0]', u'A')
('ospf3[1]', u'D')
('ospf3[2]', u'C')
('br[0]', u'G')
('br[1]', u'F')
('rip1[0]', u'H')
('rip1[1]', u'G')
('rip2[0]', u'B')
('rip2[1]', u'H')


In [6]:
config['dummysection'][keys[0]]

u'A'

In [22]:
# we only need unique links
links = set(links)

In [23]:
links

{u'A', u'B', u'C', u'D', u'E', u'F', u'G', u'H'}

In [7]:
# helper functions for natural sorting
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

# sort the keys so that we keep the order of the interfaces
keys.sort(key=natural_keys)

In [8]:
keys

['br[0]',
 'br[1]',
 'host[0]',
 'ospf1[0]',
 'ospf1[1]',
 'ospf1[2]',
 'ospf2[0]',
 'ospf2[1]',
 'ospf3[0]',
 'ospf3[1]',
 'ospf3[2]',
 'rip1[0]',
 'rip1[1]',
 'rip2[0]',
 'rip2[1]',
 'server[0]']

In [9]:
m_keys

['lab_description', 'lab_version', 'lab_author', 'lab_email']

In [11]:
# we then get a dictionary of machines ignoring interfaces that have missing positions (eg: 1,3,6 instead of 0,1,2)
machines = {}
for key in keys: 
    splitted = key.split('[')
    name = splitted[0]
    splitted = splitted[1].split(']')
    ifnumber = int(splitted[0])
    if not machines.get(name):
        machines[name] = []
    if len(machines[name]) == 0 or machines[name][len(machines[name])-1][1] == ifnumber - 1:
        machines[name].append((config['dummysection'][key], ifnumber))

In [12]:
machines

{'br': [(u'G', 0), (u'F', 1)],
 'host': [(u'A', 0)],
 'ospf1': [(u'C', 0), (u'E', 1), (u'F', 2)],
 'ospf2': [(u'D', 0), (u'E', 1)],
 'ospf3': [(u'A', 0), (u'D', 1), (u'C', 2)],
 'rip1': [(u'H', 0), (u'G', 1)],
 'rip2': [(u'B', 0), (u'H', 1)],
 'server': [(u'B', 0)]}

In [13]:
metadata = {}
for m_key in m_keys:
    if config['dummysection'][m_key].startswith('"') and config['dummysection'][m_key].endswith('"'):
        config['dummysection'][m_key] = config['dummysection'][m_key][1:-1]
    metadata[m_key] = config['dummysection'][m_key]

In [14]:
metadata

{'lab_author': u'aaaaaaaaaaaaaaaa',
 'lab_description': u'absaaaaaaaaaaaaaaaaa',
 'lab_email': u'aaaaaaaaa@aaaaaaaaaaaaaa',
 'lab_version': u'1.0'}

In [ ]:
"-------------------------------------------"

In [68]:
network_block_template = '''networks: 
${links}
'''
network_link_template = '''    ${link}:
'''
service_block_template = '''version: '2.1'

services:
${services}
'''
service_template = '''    ${machine_name}:
        image: ${image_name}
        privileged: true
        container_name: netkit-${machine_name}
        ports:
            - 808${number}:80
        networks:
${links}
'''
service_link_template = '''            ${link}:
'''
# TODO lab.dep with '''depends_on: service: condition: service_healty'''

In [28]:
print(Template('''hello ${data}! 
               ''').render(data="world"))

hello world! 
               


In [36]:
conf_network = ""
for link in links:
    conf_network = conf_network + Template(network_link_template).render(link=link)

conf_network = Template(network_block_template).render(links=conf_network)

In [37]:
print(conf_network)

networks: 
    A:
    C:
    B:
    E:
    D:
    G:
    F:
    H:




In [69]:
conf_services = ""
counter = 0
for machine_name, interfaces in machines.items():
    conf_links = ""
    for link, _ in interfaces:
        conf_links = conf_links + Template(service_link_template).render(link=link)
    conf_services = conf_services + Template(service_template).render(machine_name=machine_name, image_name=IMAGE_NAME, number=counter, links=conf_links)
    counter += 1
    
conf_services = Template(service_block_template).render(services=conf_services)

In [70]:
print(conf_services)

version: '2.1'

services:
    host:
        image: quagga
        privileged: true
        container_name: netkit-host
        ports:
            - 8080:80
        networks:
            A:

    server:
        image: quagga
        privileged: true
        container_name: netkit-server
        ports:
            - 8081:80
        networks:
            B:

    rip2:
        image: quagga
        privileged: true
        container_name: netkit-rip2
        ports:
            - 8082:80
        networks:
            B:
            H:

    rip1:
        image: quagga
        privileged: true
        container_name: netkit-rip1
        ports:
            - 8083:80
        networks:
            H:
            G:

    br:
        image: quagga
        privileged: true
        container_name: netkit-br
        ports:
            - 8084:80
        networks:
            G:
            F:

    ospf1:
        image: quagga
        privileged: true
        container_name: netkit-ospf1
        ports: